In [ ]:
!pip install flask --break-system-packages
!pip install flask-restx  --break-system-packages

## **Example API System**

### **RESTful API Features**

#### **1. General Greeting**
- **GET `/example/hello`**
  - **Description**: Returns a generic greeting message.
  - **Response Schema/Model**:
    - `message`: string 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.

#### **2. Personalized Greeting**
- **GET `/example/greet/<string:name>`**
  - **Description**: Returns a personalized greeting message using the `name` path parameter.
  - **Path Parameters**:
    - `name` (string): The name of the person to greet.
  - **Response Schema/Model**: 
    - `name`: string,
    - `message`: string 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.

#### **3. Query-Based Greeting**
- **GET `/example/query`**
  - **Description**: Returns a greeting message based on query parameters (`name` and `age`).
  - **Query Parameters**:
    - `name` (string, optional): Name of the person. Defaults to `"Guest"`.
    - `age` (integer, optional): Age of the person.
  - **Response Schema/Model**:  
      - `message`: str, 
  - **HTTP Status Codes**:
    - `200 OK`: Successful response.
    - `400 Bad Request`: Invalid query parameters.
---

 


In [ ]:
import nest_asyncio
from flask import Flask, request
from flask_restx import Api, Resource, fields
from werkzeug.serving import run_simple

nest_asyncio.apply()

# Initialize Flask app and RESTX API
app = Flask(__name__)
api = Api(app, version="1.0", title="Flask RESTX API", description="A demonstration of Flask-RESTX with class-based routes")

# Namespace
example_ns = api.namespace('example', description='Endpoints demonstrating basic Flask-RESTX functionality')

# Models (Define the structure of expected input/output data)
hello_model = example_ns.model('Hello', {
    'message': fields.String(description="Greeting message", example="Hello, World!")
})

greet_model = example_ns.model('Greet', {
    'name': fields.String(required=True, description="Name of the person being greeted", example="Adam"),
    'message': fields.String(description="Personalized greeting message", example="Hello, Adam!")
})

query_model = example_ns.model('Query', {
    'name': fields.String(description="Name parameter from query string", example="Adam"),
    'age': fields.Integer(description="Age parameter from query string", example=30),
    'message': fields.String(description="Response message", example="Hello, Adam! You are 30 years old.")
})

# Routes as class-based resources
@example_ns.route('/hello')
class HelloWorld(Resource):
    @example_ns.marshal_with(hello_model)
    def get(self):
        """
        A simple endpoint to return a generic greeting message.
        """
        return {"message": "Hello, World!"}, 200
  

@example_ns.route('/greet/<string:name>')
@example_ns.param('name', 'Name of the person to greet')
class Greet(Resource):
    @example_ns.marshal_with(greet_model)
    def get(self, name):
        """
        Personalized greeting endpoint.
        Accepts a name as a path parameter and returns a greeting message.
        """
        return {"name": name, "message": f"Hello, {name}!"}, 200


@example_ns.route('/query')
class Query(Resource):
    @example_ns.doc(params={
        'name': {'description': 'Name parameter (optional)', 'default': 'Guest'},
        'age': {'description': 'Age parameter (optional)', 'type': 'integer'}
    })
    @example_ns.marshal_with(query_model)
    def get(self):
        """
        Query parameter demonstration endpoint.
        Accepts 'name' and 'age' as query parameters and returns a formatted greeting message.
        """
        name = request.args.get("name", "Guest")
        age = request.args.get("age")

        # Check if 'age' is provided and construct the message accordingly
        if age:
            return {"name": name, "age": int(age), "message": f"Hello, {name}! You are {age} years old."}, 200

        return {"name": name, "message": f"Hello, {name}!"}, 200


# Add namespace to the API
api.add_namespace(example_ns)

# Run Flask in Jupyter Notebook or standalone
run_simple('localhost', 4000, app)


## **Comments API System**

### **RESTful API Features**

#### **1. Create Comment**
- **POST `/comment`**
  - **Description**: Creates a new comment linked to a post.
  - **Request Schema/Model**:
    - `id`: integer (required) - Unique ID of the comment.
    - `content`: string (required) - Content of the comment.
    - `post_id`: integer (required) - ID of the associated post.
  - **Response Schema/Model**:
    - `success`: string
    - `data`: object
      - `id`: integer
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `201 Created`: Comment successfully created.
    - `400 Bad Request`: Invalid input.
    - `409 Conflict`: Comment with this ID already exists.

#### **2. Get Comment**
- **GET `/comment/<int:comment_id>`**
  - **Description**: Retrieves a comment by its ID.
  - **Path Parameters**:
    - `comment_id`: integer (required) - Unique ID of the comment to retrieve.
  - **Response Schema/Model**:
    - `id`: integer
    - `comment`: object
      - `id`: integer
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `200 OK`: Comment successfully retrieved.
    - `404 Not Found`: Comment not found.

#### **3. Update Comment**
- **PUT `/comment/<int:comment_id>`**
  - **Description**: Updates the content and associated post ID of an existing comment.
  - **Path Parameters**:
    - `comment_id`: integer (required) - Unique ID of the comment to update.
  - **Request Schema/Model**:
    - `content`: string (required) - Updated content of the comment.
    - `post_id`: integer (required) - Updated ID of the associated post.
  - **Response Schema/Model**:
    - `id`: integer
    - `comment`: object
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `200 OK`: Comment successfully updated.
    - `400 Bad Request`: Invalid input.
    - `404 Not Found`: Comment not found.

#### **4. Delete Comment**
- **DELETE `/comment/<int:comment_id>`**
  - **Description**: Deletes a comment by its ID.
  - **Path Parameters**:
    - `comment_id`: integer (required) - Unique ID of the comment to delete.
  - **Response Schema/Model**:
    - `success`: string
  - **HTTP Status Codes**:
    - `204 No Content`: Comment successfully deleted.
    - `404 Not Found`: Comment not found.

---

### **Models**
#### **Comment Data**
- `id` (int): Unique identifier for the comment.
- `content` (str): Content of the comment.
- `post_id` (int): ID of the associated post.

---



In [ ]:
import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask
from flask_restx import Api, Namespace, Resource, fields
from http import HTTPStatus

nest_asyncio.apply()

# Initialize Flask app and API
app = Flask(__name__)

# Wrap the initialized app with Flask-Restx swagger
api  = Api(app, version="1.0", title="Comments API System", description="A Flask-RESTX API for managing comments.")

# In-memory storage
comments = {}
""" 
 {
     "?": { }
 }
"""

# Initialize name space for grouping endpoints
comments_ns = api.namespace('comments', description="Operations related to comments")
 
# commentUpdateRequest Model 
comment_update_request_model = api.model('commentUpdateRequest',{                            
        'content': fields.String(description="Content of the comment"),
        'post_id': fields.Integer(description="ID of the associated post")
})

# commentData Model
comment_data_model =  api.model('CommentData',
    {             
        'id': fields.Integer(required=True, description="Unique ID of the comment"),
        'content': fields.String(required=True, description="Content of the comment"),
        'post_id': fields.Integer(required=True, description="ID of the associated post")
    }
)

# CommentResponse Model
comment_response_model =  comments_ns.model('CommentResponse', {
    'success': fields.String(description="Status of the operation"),
    "data": fields.Nested(comment_data_model)
   
})

# Define Resources
@comments_ns.route("/") # post
class CommentsCollection(Resource):
    
    @comments_ns.expect(comment_data_model, validate=True)
    @comments_ns.response(HTTPStatus.CREATED, "Comment successfully created", comment_response_model)
    @comments_ns.response(HTTPStatus.BAD_REQUEST, "Invalid input")
    @comments_ns.response(HTTPStatus.CONFLICT, "Comment with this ID already exists" )
    def post(self):
        """Creates a new comment linked to a post"""
        data = api.payload
        comment_id = data["id"] 
        
        # if "id" not in data or "content" not in data or "post_id" not in data:
        #     response_data = {"error", "Invalid input"}
        #     response_status_code = HTTPStatus.BAD_REQUEST
            
        #     return response_data, response_status_code
        #     return {"error", "Invalid input"}, HTTPStatus.BAD_REQUEST
            
        if comment_id in comments:
            response_data = {"error", "Comment with this ID already exists"}
            response_status_code = HTTPStatus.CONFLICT
            
            return response_data, response_status_code
            # return {"error", "Comment with this ID already exists"}, HTTPStatus.CONFLICT     
        
        comments[comment_id] = data  
    
        response_data = {"success": "Comment created", "data": data},
        response_status_code = HTTPStatus.CREATED
            
        return response_data, response_status_code
        # return {"success": "Comment created", "data": data}, HTTPStatus.CREATED
            
        

@comments_ns.route("/<int:comment_id>")
class CommentResource(Resource):
    
    @comments_ns.response(HTTPStatus.OK, "Comment successfully retrieved", comment_data_model)
    @comments_ns.response(HTTPStatus.NOT_FOUND, "Comment not found")
    def get(self, comment_id: int):
        """Retrieves a comment by its ID"""
        if comment_id not in comments:
            response_data = {"error": "Comment not found"}
            response_status_code = HTTPStatus.NOT_FOUND
            
            return response_data, response_status_code
            # return {"error": "Comment not found"}, HTTPStatus.NOT_FOUND

        response_data = comments[comment_id]
        response_status_code = HTTPStatus.OK
        
        return response_data, response_status_code
        # return {"id": comment_id, "comment": comments[comment_id]}, HTTPStatus.OK


            
    @comments_ns.expect(comment_update_request_model, validate=True)
    @comments_ns.response(HTTPStatus.OK, "Comment successfully updated", comment_data_model)
    @comments_ns.response(HTTPStatus.BAD_REQUEST, "Invalid input")
    @comments_ns.response(HTTPStatus.NOT_FOUND, "Comment not found")
    def put(self, comment_id:int):
        """Update an existing comment"""
        if comment_id not in comments:
            response_data = {"error": "Comment not found"}
            response_status_code = HTTPStatus.NOT_FOUND
            
            return response_data, response_status_code
            # return {"error": "Comment not found"}, HTTPStatus.NOT_FOUND
            
        
        """ 
        comment_id=1
        data = {
            id: 1, 
            content: "123123112312312312131"
        }
        comments[comment_id]  = {
            id:1
            content: "12312312"
            post_id: 11
        }
        **comments[comment_id] = id:1, content: "12312312", ost_id: 11
        **data = id: 1, content: "123123112312312312131"
        comments[comment_id]  = {**comments[comment_id], **data } = {id:1, content: "12312312", ost_id: 11, id: 1, content: "123123112312312312131"}
        now clear with override
        comments[comment_id]  = {**comments[comment_id], **data } = { ost_id: 11, id: 1, content: "123123112312312312131"}
    
        """
        
        data = api.payload
        comments[comment_id] = {**comments[comment_id], **data }
    
        response_data =  data
        response_status_code = HTTPStatus.OK
        
        return response_data, response_status_code

    @comments_ns.response(HTTPStatus.NO_CONTENT, "Comment successfully deleted")
    @comments_ns.response(HTTPStatus.NOT_FOUND, "Comment not found")
    def delete(self, comment_id:int):
        """Delete a comment by ID"""
        if comment_id not in comments:
            response_data = {"error": "Comment not found"}
            response_status_code = HTTPStatus.NOT_FOUND
            
            return response_data, response_status_code
            # return {"error": "Comment not found"}, HTTPStatus.NOT_FOUND
        
        del comments[comment_id]
        
        response_data =  {"success": "Comment deleted"}
        response_status_code = HTTPStatus.NO_CONTENT
        
        return response_data, response_status_code
        # return {"success": "Comment deleted"}, HTTPStatus.NO_CONTENT
        
    
# add name space to API
api.add_namespace(comments_ns, path="/comment")

run_simple("localhost", 5000, app)

## **Posts Management API System**

### **RESTful API Features**

#### **1. Create Post**
- **POST `/posts`**
  - **Description**: Creates a new post with a unique ID, title, and content.
  - **Request Schema/Model**:
    - `id`: integer (required)
    - `title`: string (required)
    - `content`: string (required)
  - **Response Schema/Model**:
    - `success`: string
    - `data`: object
      - `id`: integer
      - `title`: string
      - `content`: string
  - **HTTP Status Codes**:
    - `201 Created`: Post successfully created.
    - `400 Bad Request`: Invalid input.
    - `409 Conflict`: Post with this ID already exists.

#### **2. Get Post**
- **GET `/posts/<int:post_id>`**
  - **Description**: Retrieves a post by its ID.
  - **Path Parameters**:
    - `post_id`: integer (required)
  - **Response Schema/Model**:
    - `id`: integer
    - `post`: object
      - `id`: integer
      - `title`: string
      - `content`: string
  - **HTTP Status Codes**:
    - `200 OK`: Post successfully retrieved.
    - `404 Not Found`: Post not found.

#### **3. Update Post**
- **PUT `/posts/<int:post_id>`**
  - **Description**: Updates the title and content of an existing post by its ID.
  - **Path Parameters**:
    - `post_id`: integer (required)
  - **Request Schema/Model**:
    - `title`: string (required)
    - `content`: string (required)
  - **Response Schema/Model**:
    - `id`: integer
    - `post`: object
      - `title`: string
      - `content`: string
  - **HTTP Status Codes**:
    - `200 OK`: Post successfully updated.
    - `400 Bad Request`: Invalid input.
    - `404 Not Found`: Post not found.

#### **4. Delete Post**
- **DELETE `/posts/<int:post_id>`**
  - **Description**: Deletes a post by its ID.
  - **Path Parameters**:
    - `post_id`: integer (required)
  - **Response Schema/Model**:
    - `success`: string
  - **HTTP Status Codes**:
    - `204 No Content`: Post successfully deleted.
    - `404 Not Found`: Post not found.

---

### **Models**
#### **Data (list of dictionary | database table in the form of ORM)**
- `id` (int): Unique identifier for the post.
- `title` (str): Title of the post.
- `content` (str): Content of the post.

---
 



In [ ]:
import nest_asyncio

from flask import Flask
from flask_restx import Api, Resource, fields
from http import HTTPStatus
from werkzeug.serving import run_simple

nest_asyncio.apply()

app = Flask(__name__)
api = Api(app, version="1.0", title="Posts Management API System", description="A Flask-RESTX API for managing posts.")

# Namespace
post_ns = api.namespace('posts', description="Post management operations")

# Models
create_post_model = post_ns.model('CreatePost', {
    'id': fields.Integer(required=True, description="Unique ID of the post", example=1),
    'title': fields.String(required=True, description="Title of the post", example="My First Post"),
    'content': fields.String(required=True, description="Content of the post", example="This is the content of my first post.")
})

post_response_model = post_ns.model('PostResponse', {
    'id': fields.Integer(description="Unique ID of the post", example=1),
    'title': fields.String(description="Title of the post", example="My First Post"),
    'content': fields.String(description="Content of the post", example="This is the content of my first post.")
})

delete_response_model = post_ns.model('DeleteResponse', {
    'success': fields.String(description="Success message", example="Post deleted")
})

# In-memory data store
posts = {}

# Resources
@post_ns.route("/")
class PostsCollection(Resource):
    @post_ns.expect(create_post_model, validate=True)
    @post_ns.response(HTTPStatus.CREATED, "Post successfully created", post_response_model)
    @post_ns.response(HTTPStatus.BAD_REQUEST, "Invalid input")
    @post_ns.response(HTTPStatus.CONFLICT, "Post with this ID already exists")
    def post(self):
        """Create a new post"""
        data = api.payload
        post_id = data['id']

        if post_id in posts:
            return {"error": "Post with this ID already exists"}, HTTPStatus.CONFLICT

        posts[post_id] = data
        return {"success": "Post created", "data": data}, HTTPStatus.CREATED


@post_ns.route("/<int:post_id>")
class PostResource(Resource):
    @post_ns.response(HTTPStatus.OK, "Post successfully retrieved", post_response_model)
    @post_ns.response(HTTPStatus.NOT_FOUND, "Post not found")
    def get(self, post_id):
        """Retrieve a post by ID"""
        if post_id not in posts:
            return {"error": "Post not found"}, HTTPStatus.NOT_FOUND

        return {"id": post_id, "post": posts[post_id]}, HTTPStatus.OK

    @post_ns.expect(create_post_model, validate=True)
    @post_ns.response(HTTPStatus.OK, "Post successfully updated", post_response_model)
    @post_ns.response(HTTPStatus.BAD_REQUEST, "Invalid input")
    @post_ns.response(HTTPStatus.NOT_FOUND, "Post not found")
    def put(self, post_id):
        """Update an existing post"""
        if post_id not in posts:
            return {"error": "Post not found"}, HTTPStatus.NOT_FOUND

        data = api.payload
        posts[post_id] = {"id": post_id, "title": data['title'], "content": data['content']}
        return {"id": post_id, "post": posts[post_id]}, HTTPStatus.OK

    @post_ns.response(HTTPStatus.NO_CONTENT, "Post successfully deleted")
    @post_ns.response(HTTPStatus.NOT_FOUND, "Post not found")
    def delete(self, post_id):
        """Delete a post by ID"""
        if post_id not in posts:
            return {"error": "Post not found"}, HTTPStatus.NOT_FOUND

        del posts[post_id]
        return {"success": "Post deleted"}, HTTPStatus.NO_CONTENT


# Add namespace to API
api.add_namespace(post_ns)
 
run_simple('localhost', 4000, app)


## **Posts and Comments API System**

### **RESTful API Features**

#### **Posts Management**

##### **1. Create Post**
- **POST `/post`**
  - **Description**: Creates a new post with a unique ID, title, and content.
  - **Request Schema/Model**:
    - `id`: integer (required)
    - `title`: string (required)
    - `content`: string (required)
  - **Response Schema/Model**:
    - `success`: string
    - `data`: object
      - `id`: integer
      - `title`: string
      - `content`: string
  - **HTTP Status Codes**:
    - `201 Created`: Post successfully created.
    - `400 Bad Request`: Invalid input.
    - `409 Conflict`: Post with this ID already exists.

##### **2. Get Post**
- **GET `/post/<int:post_id>`**
  - **Description**: Retrieves a post by its ID.
  - **Path Parameters**:
    - `post_id`: integer (required)
  - **Response Schema/Model**:
    - `id`: integer
    - `post`: object
      - `id`: integer
      - `title`: string
      - `content`: string
  - **HTTP Status Codes**:
    - `200 OK`: Post successfully retrieved.
    - `404 Not Found`: Post not found.

##### **3. Update Post**
- **PUT `/post/<int:post_id>`**
  - **Description**: Updates the title and content of an existing post by its ID.
  - **Path Parameters**:
    - `post_id`: integer (required)
  - **Request Schema/Model**:
    - `title`: string (required)
    - `content`: string (required)
  - **Response Schema/Model**:
    - `id`: integer
    - `post`: object
      - `title`: string
      - `content`: string
  - **HTTP Status Codes**:
    - `200 OK`: Post successfully updated.
    - `400 Bad Request`: Invalid input.
    - `404 Not Found`: Post not found.

##### **4. Delete Post**
- **DELETE `/post/<int:post_id>`**
  - **Description**: Deletes a post by its ID.
  - **Path Parameters**:
    - `post_id`: integer (required)
  - **Response Schema/Model**:
    - `success`: string
  - **HTTP Status Codes**:
    - `204 No Content`: Post successfully deleted.
    - `404 Not Found`: Post not found.

#### **Comments Management**

##### **1. Create Comment**
- **POST `/comment`**
  - **Description**: Creates a new comment linked to a post.
  - **Request Schema/Model**:
    - `id`: integer (required)
    - `content`: string (required)
    - `post_id`: integer (required)
  - **Response Schema/Model**:
    - `success`: string
    - `data`: object
      - `id`: integer
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `201 Created`: Comment successfully created.
    - `400 Bad Request`: Invalid input.
    - `409 Conflict`: Comment with this ID already exists.

##### **2. Get Comment**
- **GET `/comment/<int:comment_id>`**
  - **Description**: Retrieves a comment by its ID.
  - **Path Parameters**:
    - `comment_id`: integer (required)
  - **Response Schema/Model**:
    - `id`: integer
    - `comment`: object
      - `id`: integer
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `200 OK`: Comment successfully retrieved.
    - `404 Not Found`: Comment not found.

##### **3. Update Comment**
- **PUT `/comment/<int:comment_id>`**
  - **Description**: Updates the content of an existing comment by its ID.
  - **Path Parameters**:
    - `comment_id`: integer (required)
  - **Request Schema/Model**:
    - `content`: string (required)
    - `post_id`: integer (required)
  - **Response Schema/Model**:
    - `id`: integer
    - `comment`: object
      - `content`: string
      - `post_id`: integer
  - **HTTP Status Codes**:
    - `200 OK`: Comment successfully updated.
    - `400 Bad Request`: Invalid input.
    - `404 Not Found`: Comment not found.

##### **4. Delete Comment**
- **DELETE `/comment/<int:comment_id>`**
  - **Description**: Deletes a comment by its ID.
  - **Path Parameters**:
    - `comment_id`: integer (required)
  - **Response Schema/Model**:
    - `success`: string
  - **HTTP Status Codes**:
    - `204 No Content`: Comment successfully deleted.
    - `404 Not Found`: Comment not found.

---

### **Models**
#### **Post Data**
- `id` (int): Unique identifier for the post.
- `title` (str): Title of the post.
- `content` (str): Content of the post.

#### **Comment Data**
- `id` (int): Unique identifier for the comment.
- `content` (str): Content of the comment.
- `post_id` (int): ID of the associated post.

---



In [ ]:
import nest_asyncio
from werkzeug.serving import run_simple
from flask import Flask
from flask_restx import Api, Namespace, Resource, fields

nest_asyncio.apply()

# Initialize Flask app
app = Flask(__name__)
api = Api(app, version="1.0", title="Posts and Comments API System", description="A Flask-RESTX API for managing posts and comments.")

# In-memory storage
posts = {}
comments = {}

# Namespaces
posts_ns = Namespace('posts', description="Operations related to posts")
comments_ns = Namespace('comments', description="Operations related to comments")

# Models
post_model = posts_ns.model('Post', {
    'id': fields.Integer(required=True, description="Unique ID of the post"),
    'title': fields.String(required=True, description="Title of the post"),
    'content': fields.String(required=True, description="Content of the post")
})

post_response_model = posts_ns.model('PostResponse', {
    'id': fields.Integer(description="Unique ID of the post"),
    'title': fields.String(description="Title of the post"),
    'content': fields.String(description="Content of the post")
})

comment_model = comments_ns.model('Comment', {
    'id': fields.Integer(required=True, description="Unique ID of the comment"),
    'content': fields.String(required=True, description="Content of the comment"),
    'post_id': fields.Integer(required=True, description="ID of the associated post")
})

comment_response_model = comments_ns.model('CommentResponse', {
    'id': fields.Integer(description="Unique ID of the comment"),
    'content': fields.String(description="Content of the comment"),
    'post_id': fields.Integer(description="ID of the associated post")
})

# Posts Namespace
@posts_ns.route("/")
class Posts(Resource):
    @posts_ns.expect(post_model, validate=True)
    @posts_ns.response(201, "Post successfully created", post_response_model)
    @posts_ns.response(400, "Invalid input")
    @posts_ns.response(409, "Post with this ID already exists")
    def post(self):
        """Create a new post"""
        data = api.payload
        post_id = data['id']

        if post_id in posts:
            return {"error": "Post with this ID already exists"}, 409

        posts[post_id] = data
        return {"success": "Post created", "data": data}, 201

@posts_ns.route("/<int:post_id>")
class Post(Resource):
    @posts_ns.response(200, "Post retrieved successfully", post_response_model)
    @posts_ns.response(404, "Post not found")
    def get(self, post_id):
        """Retrieve a post by ID"""
        if post_id not in posts:
            return {"error": "Post not found"}, 404

        return {"id": post_id, "post": posts[post_id]}, 200

    @posts_ns.expect(post_model, validate=True)
    @posts_ns.response(200, "Post successfully updated", post_response_model)
    @posts_ns.response(400, "Invalid input")
    @posts_ns.response(404, "Post not found")
    def put(self, post_id):
        """Update an existing post"""
        if post_id not in posts:
            return {"error": "Post not found"}, 404

        data = api.payload
        posts[post_id] = {"id": post_id, "title": data['title'], "content": data['content']}
        return {"id": post_id, "post": posts[post_id]}, 200

    @posts_ns.response(204, "Post successfully deleted")
    @posts_ns.response(404, "Post not found")
    def delete(self, post_id):
        """Delete a post by ID"""
        if post_id not in posts:
            return {"error": "Post not found"}, 404

        del posts[post_id]
        return {"success": "Post deleted"}, 204

# Comments Namespace
@comments_ns.route("/")
class Comments(Resource):
    @comments_ns.expect(comment_model, validate=True)
    @comments_ns.response(201, "Comment successfully created", comment_response_model)
    @comments_ns.response(400, "Invalid input")
    @comments_ns.response(409, "Comment with this ID already exists")
    def post(self):
        """Create a new comment"""
        data = api.payload
        comment_id = data['id']

        if comment_id in comments:
            return {"error": "Comment with this ID already exists"}, 409

        comments[comment_id] = data
        return {"success": "Comment created", "data": data}, 201

@comments_ns.route("/<int:comment_id>")
class Comment(Resource):
    @comments_ns.response(200, "Comment retrieved successfully", comment_response_model)
    @comments_ns.response(404, "Comment not found")
    def get(self, comment_id):
        """Retrieve a comment by ID"""
        if comment_id not in comments:
            return {"error": "Comment not found"}, 404

        return {"id": comment_id, "comment": comments[comment_id]}, 200

    @comments_ns.expect(comment_model, validate=True)
    @comments_ns.response(200, "Comment successfully updated", comment_response_model)
    @comments_ns.response(400, "Invalid input")
    @comments_ns.response(404, "Comment not found")
    def put(self, comment_id):
        """Update an existing comment"""
        if comment_id not in comments:
            return {"error": "Comment not found"}, 404

        data = api.payload
        comments[comment_id] = {"id": comment_id, "content": data['content'], "post_id": data['post_id']}
        return {"id": comment_id, "comment": comments[comment_id]}, 200

    @comments_ns.response(204, "Comment successfully deleted")
    @comments_ns.response(404, "Comment not found")
    def delete(self, comment_id):
        """Delete a comment by ID"""
        if comment_id not in comments:
            return {"error": "Comment not found"}, 404

        del comments[comment_id]
        return {"success": "Comment deleted"}, 204

# Add namespaces to API
api.add_namespace(posts_ns, path='/posts')
api.add_namespace(comments_ns, path='/comments')

 
run_simple('localhost', 4000, app)
